In [169]:
import pandas as pd
import nltk
from deep_translator import GoogleTranslator
from nltk.tokenize import word_tokenize
from string import punctuation
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
punctuation = list(punctuation)
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Всеволод\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Всеволод\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Импорт данных и предобработка

In [18]:
df = pd.read_csv("spam_or_not_spam.csv")
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2999 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   2999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [13]:
def translate_text(text):
    translator = GoogleTranslator(source='auto', target='ru')
    try:
        translation = translator.translate(text)
        return translation
    except Exception as e:
        return f"Translation error: {str(e)}"

# Функция для разделения текста на чанки и последующего объединения результатов
def translate_chunked(text, chunk_size=5000):
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    translated_chunks = [translate_text(chunk) for chunk in chunks]
    return ''.join(translated_chunks)

# Применение функции перевода к датасету
def translate_dataset(df):
    translated_emails = []

    for index, row in df.iterrows():
        email = row['email']
        translated_email = translate_chunked(email)
        translated_emails.append(translated_email)

    df['translated_email'] = translated_emails
    return df

df_translated = translate_dataset(df)
df_translated.drop(columns = ['email'], inplace = True)

,email,label,translated_email
2995,abc s good morning america ranks it the NUMBE...,1,"Доброе утро, Америка, компания Abc назвала ее ..."
2996,hyperlink hyperlink hyperlink let mortgage le...,1,гиперссылка гиперссылка гиперссылка позволит и...
2997,thank you for shopping with us gifts for all ...,1,спасибо за покупки у нас подарки на все случаи...
2998,the famous ebay marketing e course learn to s...,1,Знаменитый электронный курс по маркетингу на e...
2999,hello this is chinese traditional 子 件 NUMBER世...,1,"здравствуйте, это китайская традиционная НОМЕР..."


In [29]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Всеволод\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
def normalize_text(s):
   #Очистка от html
    soup = BeautifulSoup(s, 'html.parser')
    soup.get_text()
   #Удаление скрипта
    for data in soup(['style', 'script']):
      data.decompose()
    script_out = ' '.join(soup.stripped_strings)
  #Токенизация
    tokens = word_tokenize(s)
  # Удаляем пунктуацию
    tokens_without_punct = [i for i in tokens if i not in punctuation]
    low_tokens = [i.lower() for i in tokens_without_punct]
  # удаляем стоп-слова из нашего текста
    stopwords = nltk.corpus.stopwords.words('russian')
    words_without_stop = [i for i in low_tokens if i not in stopwords]
  # Лемматизация
    lemmatizer = nltk.WordNetLemmatizer()
    lemms = [lemmatizer.lemmatize(word) for word in words_without_stop]
  # Вывод значения в строке
    total=''
    for el in lemms:
      total+=el
      total+=' '
    return total

In [36]:
df_translated['normalized'] = df_translated['translated_email'].apply(normalize_text)

C:\Users\Всеволод\AppData\Local\Temp\ipykernel_7804\3918262440.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(s, 'html.parser')


# Выбор лучшей модели и тюнинг гиперпараметров

Для получения эмбеддингов будем использовать TfidfVectorizer - ведь спам очень хорошо детектится именно по ключевым словам

In [101]:
vectorizer = TfidfVectorizer(max_df = 0.4, max_features = 5000)
X = vectorizer.fit_transform(df_translated['normalized'])
y = df_translated['label']

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

А в качестве базовой модели используем обычную линейную регрессию - опыт показывает, что в рамках таких задач она демонстрирует достаточно высокую точность, оставаясь очень простой и высокопроизводительной моделью. Подберем гиперпараметры через обычный гридсерч.

In [114]:
l_grid_search = GridSearchCV(LogisticRegression(),
                               [{"C":np.logspace(-3,5,9), "penalty":["l1","l2"]}],
                               cv=5,
                               verbose=0)

In [ ]:
l_grid_search.fit(X_train, y_train)

In [116]:
print(l_grid_search.best_params_)
print(l_grid_search.best_score_)
print(l_grid_search.best_estimator_)

{'C': 1000.0, 'penalty': 'l2'}
0.9879114474599862
LogisticRegression(C=1000.0)


In [117]:
final_model = l_grid_search.best_estimator_

# А теперь настроим трэшхолд с учетом нашей исходной задачи

В первую очередь для нас важно добиться того, чтобы мы сохраняли высокую точность в детекции спама при условии, что мы не будем относить к спаму хорошие посты. То есть recall нулевого класа должен быть равен 1. 

Поэтому нами будет использоваться достаточно высокий трэшхолд.

In [164]:
threshold = 0.75

In [165]:
y_pred_proba = final_model.predict_proba(X_test)
y_pred = (y_pred_proba[:, 1] > threshold).astype(int)

In [166]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       500
           1       1.00      0.94      0.97       100

    accuracy                           0.99       600
   macro avg       0.99      0.97      0.98       600
weighted avg       0.99      0.99      0.99       600



In [167]:
print (confusion_matrix(y_test, y_pred))

[[500   0]
 [  6  94]]


# Обучение финальной модели и её сохранение для потомков

Обучим модель с отобранными гиперпараметрами на всем наборе данных.

In [170]:
final_model.fit(X, y)

LogisticRegression(C=1000.0)

Сохраним модель и векторайзер

In [171]:
from joblib import dump, load
dump(final_model, 'spam_clf.joblib')
dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']